In [26]:
import pandas as pd
import numpy as np
import csv
import sqlite3
from sqlalchemy import create_engine

In [2]:
#Prior to this, I ran a reverse geo lookup to get County information; then ran through Open Refine
df = pd.read_csv("Lightning by County.csv")
df.head()

,Day,CENTERLON,CENTERLAT,Count,County
0,20000101,-124.0,40.8,3,Humboldt
1,20000101,-124.0,40.9,2,Humboldt
2,20000112,-120.1,38.2,1,Tuolumne
3,20000112,-124.0,41.6,1,Del Norte
4,20000115,-121.1,38.9,1,Placer


In [3]:
#Convert to a common date format
df['Day'] = pd.to_datetime(df['Day'], format='%Y%m%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221225 entries, 0 to 221224
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Day        221225 non-null  datetime64[ns]
 1   CENTERLON  221225 non-null  float64       
 2   CENTERLAT  221225 non-null  float64       
 3   Count      221225 non-null  int64         
 4   County     221225 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 8.4+ MB


In [4]:
#Group the lightning strikes by day and county to get an overall sum and export it for data review/cleaning
grouped_df = df.groupby(["Day", "County"]).agg({"Count":"sum"})
print(grouped_df)
grouped_df.to_csv('grouped_lightning.csv')

                        Count
Day        County            
2000-01-01 Humboldt         5
2000-01-12 Del Norte        1
           Tuolumne         1
2000-01-15 Colusa           1
           Nevada           5
...                       ...
2015-12-24 Stanislaus       5
2015-12-25 Amador           5
           Fresno           1
           Kern             1
           Tulare           8

[19516 rows x 1 columns]


In [6]:
#Import drought data
drought_df = pd.read_csv("CA Drought Data.csv")
drought_df.head()

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,Weight,ValidStart,ValidEnd
0,20151229,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/29/2015,1/4/2016
1,20151222,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/22/2015,12/28/2015
2,20151215,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/15/2015,12/21/2015
3,20151208,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/8/2015,12/14/2015
4,20151201,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/1/2015,12/7/2015


In [7]:
#Change the map begin and end dates to a common date format
drought_df['MapDate'] = pd.to_datetime(drought_df['MapDate'], format='%Y%m%d')
drought_df['EndDate'] = drought_df['MapDate'] + pd.DateOffset(days=6)
drought_df.head()

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,Weight,ValidStart,ValidEnd,EndDate
0,2015-12-29,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/29/2015,1/4/2016,2016-01-04
1,2015-12-22,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/22/2015,12/28/2015,2015-12-28
2,2015-12-15,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/15/2015,12/21/2015,2015-12-21
3,2015-12-08,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/8/2015,12/14/2015,2015-12-14
4,2015-12-01,6001,Alameda,CA,0.0,0.0,0.0,0.0,81.04,18.96,418.96,12/1/2015,12/7/2015,2015-12-07


In [8]:
#Import weather data found by Sean. Sean also cleaned the data
wx_df = pd.read_csv("weather_dataset.csv")
wx_df.head()

,WEATHER_ID,COUNTY,DATE,LATITUDE,LONGITUDE,TMAX,TMIN,PRCP,TSUN
0,1,Alameda,1/1/2000,37.8744,-121.58319,51.230769,42.538462,0.000,51.230769
1,2,Alameda,1/1/2001,37.8744,-121.58319,59.692308,39.769231,0.000,59.692308
2,3,Alameda,1/1/2002,37.8744,-121.58319,59.750000,52.166667,0.073,59.750000
3,4,Alameda,1/1/2003,37.8744,-121.58319,56.181818,38.818182,0.010,56.181818
4,5,Alameda,1/1/2004,37.8744,-121.58319,52.000000,43.363636,1.135,52.000000


In [9]:
#Change date to the common format
wx_df['DATE'] = pd.to_datetime(wx_df['DATE'], infer_datetime_format=True)
wx_df.head()

,WEATHER_ID,COUNTY,DATE,LATITUDE,LONGITUDE,TMAX,TMIN,PRCP,TSUN
0,1,Alameda,2000-01-01,37.8744,-121.58319,51.230769,42.538462,0.000,51.230769
1,2,Alameda,2001-01-01,37.8744,-121.58319,59.692308,39.769231,0.000,59.692308
2,3,Alameda,2002-01-01,37.8744,-121.58319,59.750000,52.166667,0.073,59.750000
3,4,Alameda,2003-01-01,37.8744,-121.58319,56.181818,38.818182,0.010,56.181818
4,5,Alameda,2004-01-01,37.8744,-121.58319,52.000000,43.363636,1.135,52.000000


In [10]:
#Group weather by county and date
grouped_wx = wx_df.groupby(["DATE", "COUNTY"]).mean()
grouped_wx.head()

WEATHER_ID  LATITUDE  LONGITUDE       TMAX       TMIN  \
DATE       COUNTY                                                              
2000-01-01 Alameda            1.0  37.87440 -121.58319  51.230769  42.538462   
           Alpine          5845.0  38.70861 -119.61000  24.833333   9.166667   
           Amador         11689.0  38.48830 -120.32500  46.666667  33.333333   
           Butte          17533.0  40.07890 -121.38610  50.000000  32.500000   
           Calaveras      23377.0  38.73830 -120.31130  48.000000  32.333333   

                       PRCP       TSUN  
DATE       COUNTY                       
2000-01-01 Alameda      0.0  51.230769  
           Alpine       0.0  24.833333  
           Amador       0.0  46.666667  
           Butte        0.0  50.000000  
           Calaveras    0.0  48.000000

In [14]:
#Create an SQLite database in memory using the 3 dataframes created thus far
conn = sqlite3.connect(':memory:')
grouped_df.to_sql('lightning', conn, index=True)
drought_df.to_sql('drought', conn, index=False)
grouped_wx.to_sql('wx', conn, index=True)

In [15]:
#Tie all 3 databases together on County and date
qry = '''
    select  
        wx.DATE Date,
        wx.COUNTY County,
        lightning.Count Strikes,
        Weight DroughtWt,
        wx.TMAX MaxTemp,
        TMIN MinTemp,
        PRCP Precip
    from
        wx left outer join lightning on
        Date = Day and lightning.County = wx.County
        left outer join drought on 
        wx.Date between MapDate and EndDate and wx.County = drought.County
        '''
df1 = pd.read_sql_query(qry, conn)
print(df1)

                       Date      County  Strikes  DroughtWt    MaxTemp  \
0       2000-01-01 00:00:00    Alameda       NaN        NaN  51.230769   
1       2000-01-01 00:00:00     Alpine       NaN        NaN  24.833333   
2       2000-01-01 00:00:00     Amador       NaN        NaN  46.666667   
3       2000-01-01 00:00:00      Butte       NaN        NaN  50.000000   
4       2000-01-01 00:00:00  Calaveras       NaN        NaN  48.000000   
...                     ...         ...      ...        ...        ...   
330457  2015-12-31 00:00:00     Tulare       NaN     500.00  45.764706   
330458  2015-12-31 00:00:00   Tuolumne       NaN     500.00  44.125000   
330459  2015-12-31 00:00:00    Ventura       NaN     500.00  56.300000   
330460  2015-12-31 00:00:00       Yolo       NaN     490.45        NaN   
330461  2015-12-31 00:00:00       Yuba       NaN     500.00  46.666667   

          MinTemp  Precip  
0       42.538462     0.0  
1        9.166667     0.0  
2       33.333333     0.0  

In [20]:
#Get previously cleaned fire data by Ramin. Some counties were missing, so I utilized the reverse geo lookup tool and 
#filtering by class D and higher to limite the impactful fires
fire_df = pd.read_csv("FiresWithCounty.csv")
fire_df.head()

,Year,Date,Discover_Julian,Cause,Acres,Size_Class,County,Fire
0,2004,2453284.5,280,Equipment Use,16823.0,G,Amador,1
1,2004,2453291.5,287,Equipment Use,7700.0,G,El Dorado,1
2,2005,2453560.5,190,Miscellaneous,1149.0,F,Kern,1
3,2005,2453606.5,236,Campfire,124.0,D,San Diego,1
4,2005,2453629.5,259,Miscellaneous,775.0,E,Lake,1


In [21]:
#Convert the Day of Year and Year to a common format
fire_df['Date'] = pd.to_datetime(fire_df.Year, format='%Y') + pd.to_timedelta(fire_df.Discover_Julian - 1, unit='d')
fire_df.head()

,Year,Date,Discover_Julian,Cause,Acres,Size_Class,County,Fire
0,2004,2004-10-06,280,Equipment Use,16823.0,G,Amador,1
1,2004,2004-10-13,287,Equipment Use,7700.0,G,El Dorado,1
2,2005,2005-07-09,190,Miscellaneous,1149.0,F,Kern,1
3,2005,2005-08-24,236,Campfire,124.0,D,San Diego,1
4,2005,2005-09-16,259,Miscellaneous,775.0,E,Lake,1


In [143]:
#Read in the previously created mega_data.csv as a dataframe. Population information was added in Excel using INDEX and MATCH
#due to the format of the table
mega_df = pd.read_csv("mega_data.csv")
mega_df['Date'] = pd.to_datetime(mega_df['Date'], format='%Y%m%d')
mega_df.head()

,Date,County,Strikes,DroughtWt,MaxTemp,MinTemp,Precip,Year,Population
0,2000-01-01,Alameda,0,0.0,51.230769,42.538462,0.0,2000,1449840
1,2000-01-01,Alpine,0,0.0,24.833333,9.166667,0.0,2000,1209
2,2000-01-01,Amador,0,0.0,46.666667,33.333333,0.0,2000,35153
3,2000-01-01,Butte,0,0.0,50.000000,32.500000,0.0,2000,203807
4,2000-01-01,Calaveras,0,0.0,48.000000,32.333333,0.0,2000,40645


In [144]:
#Read in previously created fire.csv as a dataframe. Fire had a column added for 1 to represent a fire had started
fire_df = pd.read_csv("fire.csv")
fire_df['Date'] = pd.to_datetime(fire_df['Date'], format='%Y%m%d')
fire_df.head()

,Year,Date,Discover_Julian,Cause,Acres,Size_Class,County,Fire
0,2004,2004-10-06,280,Equipment Use,16823.0,G,Amador,1
1,2004,2004-10-13,287,Equipment Use,7700.0,G,El Dorado,1
2,2005,2005-07-09,190,Miscellaneous,1149.0,F,Kern,1
3,2005,2005-08-24,236,Campfire,124.0,D,San Diego,1
4,2005,2005-09-16,259,Miscellaneous,775.0,E,Lake,1


In [145]:
#Create another SQLite connection to combine fire data and mega_df
conn = sqlite3.connect(':memory:')
mega_df.to_sql('mega', conn, index=False)
fire_df.to_sql('fire', conn, index=False)

In [148]:
#Tie the 2 databases together based on date and county
qry = '''
    select  
        mega.Date Date,
        mega.County County,
        Strikes Strikes,
        DroughtWt DroughtWt,
        MaxTemp MaxTemp,
        MinTemp MinTemp,
        Precip Precip,
        Population Pop,
        Fire Fire
    from
        mega left outer join fire on
        mega.Date = fire.Date and mega.County = fire.County
        '''
df1 = pd.read_sql_query(qry, conn)
print(df1)

                       Date     County  Strikes  DroughtWt    MaxTemp  \
0       2000-01-01 00:00:00    Alameda        0       0.00  51.230769   
1       2000-01-01 00:00:00     Alpine        0       0.00  24.833333   
2       2000-01-01 00:00:00     Amador        0       0.00  46.666667   
3       2000-01-01 00:00:00      Butte        0       0.00  50.000000   
4       2000-01-01 00:00:00  Calaveras        0       0.00  48.000000   
...                     ...        ...      ...        ...        ...   
330808  2015-12-31 00:00:00     Tulare        0     500.00  45.764706   
330809  2015-12-31 00:00:00   Tuolumne        0     500.00  44.125000   
330810  2015-12-31 00:00:00    Ventura        0     500.00  56.300000   
330811  2015-12-31 00:00:00       Yolo        0     490.45        NaN   
330812  2015-12-31 00:00:00       Yuba        0     500.00  46.666667   

          MinTemp  Precip      Pop  Fire  
0       42.538462     0.0  1449840   NaN  
1        9.166667     0.0     1209   

In [149]:
#Export the final dataframe to our consolidated data.csv file
df1.to_csv('data.csv')